In [52]:
import pandas as pd
import numpy as np
import re
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 4000)
%matplotlib inline

# Startup Success Analysis

## I. Data Cleaning

In [53]:
data = pd.read_csv("data/investments_VC.csv", encoding="latin-1")

In [54]:
data.head()

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,founded_month,founded_quarter,founded_year,first_funding_at,last_funding_at,seed,venture,equity_crowdfunding,undisclosed,convertible_note,debt_financing,angel,grant,private_equity,post_ipo_equity,post_ipo_debt,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H
0,/organization/waywire,#waywire,http://www.waywire.com,|Entertainment|Politics|Social Media|News|,News,"17,50,000",acquired,USA,NY,New York City,New York,1.0,2012-06-01,2012-06,2012-Q2,2012.0,2012-06-30,2012-06-30,1750000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,/organization/tv-communications,&TV Communications,http://enjoyandtv.com,|Games|,Games,"40,00,000",operating,USA,CA,Los Angeles,Los Angeles,2.0,NaN,NaN,NaN,NaN,2010-06-04,2010-09-23,0.0,4000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,/organization/rock-your-paper,'Rock' Your Paper,http://www.rockyourpaper.org,|Publishing|Education|,Publishing,"40,000",operating,EST,NaN,Tallinn,Tallinn,1.0,2012-10-26,2012-10,2012-Q4,2012.0,2012-08-09,2012-08-09,40000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,/organization/in-touch-network,(In)Touch Network,http://www.InTouchNetwork.com,|Electronics|Guides|Coffee|Restaurants|Music|i...,Electronics,"15,00,000",operating,GBR,NaN,London,London,1.0,2011-04-01,2011-04,2011-Q2,2011.0,2011-04-01,2011-04-01,1500000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,/organization/r-ranch-and-mine,-R- Ranch and Mine,NaN,|Tourism|Entertainment|Games|,Tourism,"60,000",operating,USA,TX,Dallas,Fort Worth,2.0,2014-01-01,2014-01,2014-Q1,2014.0,2014-08-17,2014-09-26,0.0,0.0,60000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Steps

1. Filter for USA ✓
2. Clean column names ✓
3. Handle missing values ✓
4. Cast data types ✓
5. Check plausibility ✓

**1. Filter for USA**

In [55]:
data = data[data["country_code"] == "USA"].reset_index(drop=True)

In [56]:
data.head()

,permalink,name,homepage_url,category_list,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,founded_at,founded_month,founded_quarter,founded_year,first_funding_at,last_funding_at,seed,venture,equity_crowdfunding,undisclosed,convertible_note,debt_financing,angel,grant,private_equity,post_ipo_equity,post_ipo_debt,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H
0,/organization/waywire,#waywire,http://www.waywire.com,|Entertainment|Politics|Social Media|News|,News,"17,50,000",acquired,USA,NY,New York City,New York,1.0,2012-06-01,2012-06,2012-Q2,2012.0,2012-06-30,2012-06-30,1750000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,/organization/tv-communications,&TV Communications,http://enjoyandtv.com,|Games|,Games,"40,00,000",operating,USA,CA,Los Angeles,Los Angeles,2.0,NaN,NaN,NaN,NaN,2010-06-04,2010-09-23,0.0,4000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,/organization/r-ranch-and-mine,-R- Ranch and Mine,NaN,|Tourism|Entertainment|Games|,Tourism,"60,000",operating,USA,TX,Dallas,Fort Worth,2.0,2014-01-01,2014-01,2014-Q1,2014.0,2014-08-17,2014-09-26,0.0,0.0,60000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,/organization/club-domains,.Club Domains,http://nic.club/,|Software|,Software,"70,00,000",NaN,USA,FL,Ft. Lauderdale,Oakland Park,1.0,2011-10-10,2011-10,2011-Q4,2011.0,2013-05-31,2013-05-31,0.0,7000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7000000.0,0.0,0.0,0.0,0.0,0.0,0.0
4,/organization/004-technologies,004 Technologies,http://004gmbh.de/en/004-interact,|Software|,Software,-,operating,USA,IL,"Springfield, Illinois",Champaign,1.0,2010-01-01,2010-01,2010-Q1,2010.0,2014-07-24,2014-07-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**2. Clean column names**

In [57]:
data.columns = [col.strip() for col in data.columns]

**3. Handle missing values**

In [58]:
data.isna().sum()

permalink                  0
name                       0
homepage_url            2239
category_list           1761
market                  1764
funding_total_usd          0
status                   560
country_code               0
state_code                26
region                     0
city                      63
funding_rounds             0
founded_at              5097
founded_month           5158
founded_quarter         5158
founded_year            5158
first_funding_at           0
last_funding_at            0
seed                       0
venture                    0
equity_crowdfunding        0
undisclosed                0
convertible_note           0
debt_financing             0
angel                      0
grant                      0
private_equity             0
post_ipo_equity            0
post_ipo_debt              0
secondary_market           0
product_crowdfunding       0
round_A                    0
round_B                    0
round_C                    0
round_D       

- drop columns `homepage_url`, `category_list`
- drop rows with missing `status`

In [59]:
df = data.copy()

In [60]:
# drop columns
df = df.drop(columns=["homepage_url", "category_list", "founded_at","founded_month","founded_quarter","founded_year"])

In [61]:
# drop rows where we have NaN status
df = df[~df["status"].isna()].reset_index(drop=True)

In [62]:
# drop rows where we have NaN markets
df = df[~df["market"].isna()].reset_index(drop=True)

In [63]:
# drop rows where we have NaN state_code
df = df[~df["state_code"].isna()].reset_index(drop=True)

In [64]:
df.isna().sum()

permalink                0
name                     0
market                   0
funding_total_usd        0
status                   0
country_code             0
state_code               0
region                   0
city                    33
funding_rounds           0
first_funding_at         0
last_funding_at          0
seed                     0
venture                  0
equity_crowdfunding      0
undisclosed              0
convertible_note         0
debt_financing           0
angel                    0
grant                    0
private_equity           0
post_ipo_equity          0
post_ipo_debt            0
secondary_market         0
product_crowdfunding     0
round_A                  0
round_B                  0
round_C                  0
round_D                  0
round_E                  0
round_F                  0
round_G                  0
round_H                  0
dtype: int64

- Handle remaining missing values
- Document every step
- Handle one column after the other

Goals: Dataframe with 0 NaNs!

In [65]:
df.dtypes

permalink                object
name                     object
market                   object
funding_total_usd        object
status                   object
country_code             object
state_code               object
region                   object
city                     object
funding_rounds          float64
first_funding_at         object
last_funding_at          object
seed                    float64
venture                 float64
equity_crowdfunding     float64
undisclosed             float64
convertible_note        float64
debt_financing          float64
angel                   float64
grant                   float64
private_equity          float64
post_ipo_equity         float64
post_ipo_debt           float64
secondary_market        float64
product_crowdfunding    float64
round_A                 float64
round_B                 float64
round_C                 float64
round_D                 float64
round_E                 float64
round_F                 float64
round_G 

***Removing unsupported dates and converting first_funding_at and last_funding_at to datetime***

In [66]:
df = df[
    df.first_funding_at.str.match(r'(19)|(20)')]
df = df[
    df.last_funding_at.str.match(r'(19)|(20)')]
df.shape
# Converting first_funding_at and last_funding_at to datetime
df.first_funding_at = pd.to_datetime(
    df.loc[:, 'first_funding_at'])
df.last_funding_at = pd.to_datetime(
    df.loc[:, 'last_funding_at'])
df.shape

(26526, 33)

In [67]:
df.dtypes

permalink                       object
name                            object
market                          object
funding_total_usd               object
status                          object
country_code                    object
state_code                      object
region                          object
city                            object
funding_rounds                 float64
first_funding_at        datetime64[ns]
last_funding_at         datetime64[ns]
seed                           float64
venture                        float64
equity_crowdfunding            float64
undisclosed                    float64
convertible_note               float64
debt_financing                 float64
angel                          float64
grant                          float64
private_equity                 float64
post_ipo_equity                float64
post_ipo_debt                  float64
secondary_market               float64
product_crowdfunding           float64
round_A                  

***Cleaning and converting funding_total_usd to numeric*** 

In [68]:
# Fill '-' with NaNs to be treated as numeric
df.funding_total_usd.replace('-', np.nan, inplace=True)
# 
df['funding_total_usd'] = df['funding_total_usd'].str.replace(',', '')
df['funding_total_usd'] = df['funding_total_usd'].str.replace('-', '')
df['funding_total_usd'] = df['funding_total_usd'].str.replace(' ', '')

df['funding_total_usd'] = df['funding_total_usd'].replace(' ', '', regex=True)

df.funding_total_usd = pd.to_numeric(df.loc[:, 'funding_total_usd'])
df.funding_rounds = pd.to_numeric(df.loc[:, 'funding_rounds'])

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26526 entries, 0 to 26526
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   permalink             26526 non-null  object        
 1   name                  26526 non-null  object        
 2   market                26526 non-null  object        
 3   funding_total_usd     22903 non-null  float64       
 4   status                26526 non-null  object        
 5   country_code          26526 non-null  object        
 6   state_code            26526 non-null  object        
 7   region                26526 non-null  object        
 8   city                  26493 non-null  object        
 9   funding_rounds        26526 non-null  float64       
 10  first_funding_at      26526 non-null  datetime64[ns]
 11  last_funding_at       26526 non-null  datetime64[ns]
 12  seed                  26526 non-null  float64       
 13  venture         

- check missing values in `funding_total_usd`: find out why they are missing and if you can impute them 
- check if you can derive whether a company IPOed. If so, what are their `status`. Can you classify companies that had an IPO as 'acquired'? Or create a new class, e.g. 'exit' ('acquired' or 'IPO')



- Subset df to only have closed and companies that had an exit

In [70]:
df['status'].value_counts()

operating    22407
acquired      2687
closed        1432
Name: status, dtype: int64

In [71]:
df.isna().sum()

permalink                  0
name                       0
market                     0
funding_total_usd       3623
status                     0
country_code               0
state_code                 0
region                     0
city                      33
funding_rounds             0
first_funding_at           0
last_funding_at            0
seed                       0
venture                    0
equity_crowdfunding        0
undisclosed                0
convertible_note           0
debt_financing             0
angel                      0
grant                      0
private_equity             0
post_ipo_equity            0
post_ipo_debt              0
secondary_market           0
product_crowdfunding       0
round_A                    0
round_B                    0
round_C                    0
round_D                    0
round_E                    0
round_F                    0
round_G                    0
round_H                    0
dtype: int64

***We create a new investor_funding_total that will take into account every investment the company has received summed in a new column***

In [72]:
df['investor_funding_total']= df.iloc[:, -23:-1].sum(axis=1)

***drop rows for which we have no investor_funding_total***

In [73]:
df = df[df["investor_funding_total"] > 0]

In [74]:
df.columns

Index(['permalink', 'name', 'market', 'funding_total_usd', 'status',
       'country_code', 'state_code', 'region', 'city', 'funding_rounds',
       'first_funding_at', 'last_funding_at', 'seed', 'venture',
       'equity_crowdfunding', 'undisclosed', 'convertible_note',
       'debt_financing', 'angel', 'grant', 'private_equity', 'post_ipo_equity',
       'post_ipo_debt', 'secondary_market', 'product_crowdfunding', 'round_A',
       'round_B', 'round_C', 'round_D', 'round_E', 'round_F', 'round_G',
       'round_H', 'investor_funding_total'],
      dtype='object')

In [75]:
# Create a new column Type for our different type of stages
df.loc[(df['post_ipo_equity'] != 0.0) 
          |(df['post_ipo_debt'] 
          != 0.0), 'Type'] = 'IPO'  

In [76]:
df.loc[(df['secondary_market'] != 0.0) 
       & (df['Type'].isna()), 'Type'] = 'IPO'

In [77]:
df.loc[(df['Type'].isna()), 'Type'] = 'Pre-IPO'

In [78]:
df.loc[(df['angel'] > df['venture']) 
       & (df['angel'] >= df['undisclosed']) 
       & (df['angel'] >= df['seed']) 
       & (df['angel'] > df['convertible_note']) 
       & (df['angel'] > df['debt_financing']) 
       & (df['angel'] > df['equity_crowdfunding']) 
       & (df['angel'] > df['product_crowdfunding']) 
       & (df['angel'] > df['private_equity']), 'Major_Investor'] = 'Angel'

In [79]:
df.loc[(df['product_crowdfunding'] > df['angel']) 
       & (df['product_crowdfunding'] > df['undisclosed']) 
       & (df['product_crowdfunding'] > df['convertible_note']) 
       & (df['product_crowdfunding'] > df['debt_financing']) 
       & (df['product_crowdfunding'] > df['seed']) 
       & (df['product_crowdfunding'] > df['equity_crowdfunding']) 
       & (df['product_crowdfunding'] > df['private_equity']) 
       & (df['product_crowdfunding'] > df['venture']), 'Major_Investor'] = 'Product Crowdfunding'

In [80]:
df.loc[(df['equity_crowdfunding'] > df['angel']) 
       & (df['equity_crowdfunding'] > df['undisclosed']) 
       & (df['equity_crowdfunding'] > df['convertible_note']) 
       & (df['equity_crowdfunding'] > df['debt_financing'])
       & (df['equity_crowdfunding'] > df['product_crowdfunding']) 
       & (df['equity_crowdfunding'] > df['private_equity']) 
       & (df['equity_crowdfunding'] > df['seed']) 
       & (df['equity_crowdfunding'] > df['venture']), 'Major_Investor'] = 'Equity Crowdfunding'

In [81]:
df.loc[(df['seed'] > df['angel']) 
       & (df['seed'] > df['private_equity']) 
       & (df['seed'] >= df['undisclosed']) 
       & (df['seed'] >= df['convertible_note']) 
       & (df['seed'] >= df['debt_financing']) 
       & (df['seed'] > df['product_crowdfunding']) 
       & (df['seed'] > df['venture']), 'Major_Investor'] = 'Seed'

In [82]:
df.loc[(df['debt_financing'] > df['angel']) 
       & (df['debt_financing'] > df['undisclosed']) 
       & (df['debt_financing'] > df['private_equity']) 
       & (df['debt_financing'] > df['convertible_note']) 
       & (df['debt_financing'] > df['seed']) 
       & (df['debt_financing'] > df['product_crowdfunding']) 
       & (df['debt_financing'] > df['venture']), 'Major_Investor'] = 'Debt Financing'

In [83]:
df.loc[(df['convertible_note'] > df['angel']) 
       & (df['convertible_note'] > df['undisclosed']) 
       & (df['convertible_note'] > df['private_equity']) 
       & (df['convertible_note'] > df['debt_financing']) 
       & (df['convertible_note'] > df['seed']) 
       & (df['convertible_note'] > df['product_crowdfunding']) 
       & (df['convertible_note'] > df['venture']), 'Major_Investor'] = 'Convertible Note'

In [84]:
df.loc[(df['undisclosed'] > df['angel']) 
       & (df['undisclosed'] > df['convertible_note']) 
       & (df['undisclosed'] > df['private_equity']) 
       & (df['undisclosed'] > df['debt_financing']) 
       & (df['undisclosed'] > df['seed']) 
       & (df['undisclosed'] > df['product_crowdfunding']) 
       & (df['undisclosed'] > df['venture']), 'Major_Investor'] = 'Undisclosed'

In [85]:
df.loc[(df['venture'] >= df['angel']) 
       & (df['venture'] >= df['undisclosed']) 
       & (df['venture'] >= df['seed']) 
       & (df['venture'] >= df['convertible_note']) 
       & (df['venture'] >= df['debt_financing']) 
       & (df['venture'] >= df['equity_crowdfunding']) 
       & (df['venture'] >= df['product_crowdfunding']) 
       & (df['venture'] >= df['private_equity']), 'Major_Investor'] = 'Venture'

df.loc[(df['Major_Investor'].isna()) 
       & ((df['round_A'] != 0.0) 
       | (df['round_B'] != 0.0) 
       | (df['round_C'] != 0.0) 
       | (df['round_D'] != 0.0) 
       | (df['round_F'] != 0.0) 
       | (df['round_G'] != 0.0) 
       | (df['round_H'] != 0.0))
       & (df['venture'] >= df['undisclosed']) 
       & (df['venture'] >= df['seed']) 
       & (df['venture'] >= df['convertible_note']) 
       & (df['venture'] >= df['debt_financing']) 
       & (df['venture'] >= df['equity_crowdfunding']) 
       & (df['venture'] >= df['product_crowdfunding']) 
       & (df['venture'] >= df['private_equity']), 'Major_Investor'] = 'Venture'

In [86]:
df.loc[(df['private_equity'] >= df['angel']) 
       & (df['private_equity'] >= df['undisclosed']) 
       & (df['private_equity'] >= df['convertible_note']) 
       & (df['private_equity'] >= df['debt_financing']) 
       & (df['private_equity'] >= df['seed']) 
       & (df['private_equity'] >= df['equity_crowdfunding']) 
       & (df['private_equity'] >= df['product_crowdfunding']) 
       & (df['private_equity'] >= df['venture']), 'Major_Investor'] = 'Private Equity'

In [87]:
df.loc[(df['Major_Investor'] == "Venture")
       & (df['venture'] == 0.0)
       & (df['round_A'] == 0.0) 
       & (df['round_B'] == 0.0) 
       & (df['round_C'] == 0.0) 
       & (df['round_D'] == 0.0) 
       & (df['round_F'] == 0.0) 
       & (df['round_G'] == 0.0) 
       & (df['round_H'] == 0.0), 'Major_Investor'] = 'NaN'   

In [88]:
df

,permalink,name,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,first_funding_at,last_funding_at,seed,venture,equity_crowdfunding,undisclosed,convertible_note,debt_financing,angel,grant,private_equity,post_ipo_equity,post_ipo_debt,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H,investor_funding_total,Type,Major_Investor
0,/organization/waywire,#waywire,News,1750000.0,acquired,USA,NY,New York City,New York,1.0,2012-06-30,2012-06-30,1750000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.750000e+06,Pre-IPO,Seed
1,/organization/tv-communications,&TV Communications,Games,4000000.0,operating,USA,CA,Los Angeles,Los Angeles,2.0,2010-06-04,2010-09-23,0.0,4000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000e+06,Pre-IPO,Venture
2,/organization/r-ranch-and-mine,-R- Ranch and Mine,Tourism,60000.0,operating,USA,TX,Dallas,Fort Worth,2.0,2014-08-17,2014-09-26,0.0,0.0,60000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.000000e+04,Pre-IPO,Equity Crowdfunding
6,/organization/1-800-doctors,1-800-DOCTORS,Health and Wellness,1750000.0,operating,USA,NJ,Newark,Iselin,1.0,2011-03-02,2011-03-02,0.0,0.0,0.0,0.0,1750000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.750000e+06,Pre-IPO,Convertible Note
8,/organization/10-20-media,10-20 Media,E-Commerce,2050000.0,operating,USA,MD,Baltimore,Woodbine,4.0,2009-06-18,2011-12-28,0.0,0.0,0.0,0.0,0.0,2050000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.050000e+06,Pre-IPO,Debt Financing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26522,/organization/zynga,Zynga,Technology,866550786.0,operating,USA,CA,SF Bay Area,San Francisco,9.0,2007-01-01,2011-02-18,0.0,866550786.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15026000.0,355187000.0,490000000.0,0.0,0.0,0.0,0.0,0.0,1.726764e+09,Pre-IPO,Venture
26523,/organization/zyngenia,Zyngenia,Biotechnology,25000000.0,operating,USA,MD,"Washington, D.C.",Gaithersburg,1.0,2010-09-09,2010-09-09,0.0,25000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.000000e+07,Pre-IPO,Venture
26524,/organization/zyomyx-inc,ZYOMYX,Biotechnology,34275015.0,operating,USA,MO,MO - Other,Fremont,4.0,2010-01-29,2014-03-26,0.0,26775015.0,0.0,0.0,0.0,0.0,0.0,7500000.0,0.0,0.0,0.0,0.0,0.0,0.0,12000000.0,0.0,0.0,0.0,0.0,0.0,0.0,4.627502e+07,Pre-IPO,Venture
26525,/organization/zyrra,Zyrra,E-Commerce,1510500.0,operating,USA,MA,Boston,Cambridge,4.0,2010-10-21,2012-10-18,495000.0,140500.0,0.0,0.0,0.0,0.0,875000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.510500e+06,Pre-IPO,Angel


***We now have a cleaned dataset with no missing total funding, the different type of Stages, and the Major Investors***

In [89]:
df[df["Type"] == "IPO"].reset_index(drop=True)

,permalink,name,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,first_funding_at,last_funding_at,seed,venture,equity_crowdfunding,undisclosed,convertible_note,debt_financing,angel,grant,private_equity,post_ipo_equity,post_ipo_debt,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H,investor_funding_total,Type,Major_Investor
0,/organization/3d-systems,3D Systems,Software,1.695000e+08,operating,USA,SC,SC - Other,Rock Hill,3.0,2014-01-15,2014-10-13,0.0,19500000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,1.500000e+08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.695000e+08,IPO,Venture
1,/organization/5barz-international,5BARz International,Technology,6.100000e+06,operating,USA,WA,Seattle,Seattle,2.0,2013-11-12,2014-11-17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.100000e+06,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.100000e+06,IPO,Private Equity
2,/organization/adaptive-media,"Adaptive Medias, Inc.",Advertising,1.040000e+07,operating,USA,CA,Anaheim,Irvine,2.0,2014-09-12,2014-09-15,0.0,5200000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.200000e+06,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.040000e+07,IPO,Venture
3,/organization/advanced-cell-technology,Advanced Cell Technology,Biotechnology,5.777700e+07,operating,USA,CA,Los Angeles,Santa Monica,3.0,2009-11-12,2014-07-03,0.0,25000000.0,0.0,0.0,0.0,2777000.0,0.0,0.0,0.0,3.000000e+07,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.777700e+07,IPO,Venture
4,/organization/aerie-pharmaceuticals,Aerie Pharmaceuticals,Biotechnology,2.536150e+08,operating,USA,NJ,Newark,Bedminster,5.0,2010-09-01,2014-09-09,0.0,41364965.0,0.0,0.0,0.0,20250000.0,0.0,0.0,0.0,1.920000e+08,0.000000e+00,0.0,0.0,0.0,41364965.0,0.0,0.0,0.0,0.0,0.0,0.0,2.949799e+08,IPO,Venture
5,/organization/aeropostale,Aeropostale,Consumers,1.500000e+08,operating,USA,NY,New York City,New York,1.0,2014-05-28,2014-05-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,1.500000e+08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.500000e+08,IPO,Private Equity
6,/organization/aircell,AirCell,Messaging,2.761086e+08,operating,USA,IL,Chicago,Itasca,3.0,2010-01-21,2011-02-06,0.0,65108581.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.110000e+08,0.000000e+00,0.0,0.0,0.0,0.0,0.0,65108581.0,0.0,0.0,0.0,0.0,3.412172e+08,IPO,Venture
7,/organization/alexza-pharmaceuticals,Alexza Pharmaceuticals,Health Care,6.873031e+07,operating,USA,CA,SF Bay Area,Mountain View,4.0,2009-10-06,2014-03-19,3000000.0,20730309.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.500000e+07,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.873031e+07,IPO,Venture
8,/organization/alimera-sciences,Alimera Sciences,Health Care,1.793000e+08,operating,USA,GA,Atlanta,Alpharetta,6.0,2005-11-30,2014-04-24,0.0,66800000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.750000e+07,3.500000e+07,0.0,0.0,0.0,31800000.0,35000000.0,0.0,0.0,0.0,0.0,0.0,2.461000e+08,IPO,Venture
9,/organization/alnylam-pharmaceuticals,Alnylam Pharmaceuticals,Health Care,7.100000e+08,operating,USA,MA,Boston,Cambridge,2.0,2011-03-28,2014-01-13,0.0,10000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.000000e+08,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.100000e+08,IPO,Venture


In [90]:
df.groupby(["status", "Type"]).count()["permalink"]

status     Type   
acquired   IPO            7
           Pre-IPO     2449
closed     IPO            2
           Pre-IPO     1228
operating  IPO          191
           Pre-IPO    19026
Name: permalink, dtype: int64

- If a company has been acquired = 'acquired'
- If a company has been closed = 'closed'
- If a company is operating nbut had an IPO = 'IPO'
- else = 'operating

In [91]:
df.loc[(df['status'] == "acquired")
    ,'new_status'] = 'acquired' 

df.loc[(df['status'] == "closed")
    ,'new_status'] = 'closed'

df.loc[(df['status'] == "operating")
     & (df['Type'] == 'IPO') 
    ,'new_status'] = 'IPO'

df.loc[(df['status'] == "operating")
     & (df['Type'] != 'IPO') 
    ,'new_status'] = 'operating' 

In [92]:
df

,permalink,name,market,funding_total_usd,status,country_code,state_code,region,city,funding_rounds,first_funding_at,last_funding_at,seed,venture,equity_crowdfunding,undisclosed,convertible_note,debt_financing,angel,grant,private_equity,post_ipo_equity,post_ipo_debt,secondary_market,product_crowdfunding,round_A,round_B,round_C,round_D,round_E,round_F,round_G,round_H,investor_funding_total,Type,Major_Investor,new_status
0,/organization/waywire,#waywire,News,1750000.0,acquired,USA,NY,New York City,New York,1.0,2012-06-30,2012-06-30,1750000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.750000e+06,Pre-IPO,Seed,acquired
1,/organization/tv-communications,&TV Communications,Games,4000000.0,operating,USA,CA,Los Angeles,Los Angeles,2.0,2010-06-04,2010-09-23,0.0,4000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.000000e+06,Pre-IPO,Venture,operating
2,/organization/r-ranch-and-mine,-R- Ranch and Mine,Tourism,60000.0,operating,USA,TX,Dallas,Fort Worth,2.0,2014-08-17,2014-09-26,0.0,0.0,60000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.000000e+04,Pre-IPO,Equity Crowdfunding,operating
6,/organization/1-800-doctors,1-800-DOCTORS,Health and Wellness,1750000.0,operating,USA,NJ,Newark,Iselin,1.0,2011-03-02,2011-03-02,0.0,0.0,0.0,0.0,1750000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.750000e+06,Pre-IPO,Convertible Note,operating
8,/organization/10-20-media,10-20 Media,E-Commerce,2050000.0,operating,USA,MD,Baltimore,Woodbine,4.0,2009-06-18,2011-12-28,0.0,0.0,0.0,0.0,0.0,2050000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.050000e+06,Pre-IPO,Debt Financing,operating
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26522,/organization/zynga,Zynga,Technology,866550786.0,operating,USA,CA,SF Bay Area,San Francisco,9.0,2007-01-01,2011-02-18,0.0,866550786.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15026000.0,355187000.0,490000000.0,0.0,0.0,0.0,0.0,0.0,1.726764e+09,Pre-IPO,Venture,operating
26523,/organization/zyngenia,Zyngenia,Biotechnology,25000000.0,operating,USA,MD,"Washington, D.C.",Gaithersburg,1.0,2010-09-09,2010-09-09,0.0,25000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.000000e+07,Pre-IPO,Venture,operating
26524,/organization/zyomyx-inc,ZYOMYX,Biotechnology,34275015.0,operating,USA,MO,MO - Other,Fremont,4.0,2010-01-29,2014-03-26,0.0,26775015.0,0.0,0.0,0.0,0.0,0.0,7500000.0,0.0,0.0,0.0,0.0,0.0,0.0,12000000.0,0.0,0.0,0.0,0.0,0.0,0.0,4.627502e+07,Pre-IPO,Venture,operating
26525,/organization/zyrra,Zyrra,E-Commerce,1510500.0,operating,USA,MA,Boston,Cambridge,4.0,2010-10-21,2012-10-18,495000.0,140500.0,0.0,0.0,0.0,0.0,875000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.510500e+06,Pre-IPO,Angel,operating


In [93]:
df =df[df["new_status"] != "operating"]

In [94]:
df.to_pickle("data/cleaned_dataset")

In [98]:
df["status"].value_counts()

acquired     2456
closed       1230
operating     191
Name: status, dtype: int64